# Computing covariate residuals
This is the region extraction step for data processing pipeline for xqtl workflow, containing the generation of:
1. Molecular_phenotype per chrom within selected regions in the format APEX and tensorQTL takes

### Input
The input for this workflow is the collection of data for 1 conditions as described in the readme of this git repo
1. 1 complete molecular phenotype data
2. 1 file containing the factors to be regressed out, with a #id column documenting the name of factors, starting with "factor_", and column names = sample names

### Output
For each collection, the output is 23 sets of :
1. Sets of EXP file for selected region, suitable to be further processed and feed into susie

1 sets of
1. PCA + Factor + Covariate file


In [2]:
[global]
import os
# Work directory & output directory
parameter: wd = "./"
# The filename name for output data
parameter: container = 'gaow/twas'
# name for the analysis output
parameter: name = 'ROSMAP'
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20




In [ ]:
[Residual_Expression_1]
# Path to the input molecular phenotype data.
parameter: molecular_pheno_whole = path
# Path to the factor file 
parameter: factor = path
input: molecular_pheno_whole,covariate
output: f'{wd}/cache/{name}.mol_phe.resid.bed'
task: trunk_workers = 1, trunk_size = 1, walltime = '4h',  mem = '20G', tags = f'{step_name}_{_output[0]:bn}'
R: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout',container = container,volumes = [f'{wd:ad}:{genotype_list:ad}']
    library("dplyr")
    library("tibble")
    library("readr")
    library("modelr")
    library("purrr")
    factor = read_delim("$[_input[1]]",delim = "\t")
    pheno = read_delim("$[_input[0]]",delim = "\t")
    factor = factor%>%filter(str_detect(`#id`,"factor"))%>%select(-`#id`)%>%as.matrix()%>%t()
    pheno_id = pheno%>%select(gene_ID)
    pheno = pheno%>%select(rownames(factor))%>%as.matrix()%>%t()
    pheno_resid = .lm.fit(x = factor, y = pheno)$residuals
    pheno_output = cbind(pheno_id, pheno_resid%>%t())
    pheno_output%>%write_delim("$[_output[0]]",delim = "\t")